In [ ]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey

In [ ]:
# Set the Cosmos DB endpoint, key and database name from the resource created in the portal.
URL = 'https://contoso-sales.documents.azure.com:443/'
KEY = 'Q27VLFw9uzv37jtpufkD3kMSjyQiMBtbd2Gq5dR3chB0t0a6H8I8c6Bpk1mSl0EzHEYy0M6hgWzbACDbI6vOpA=='
client = CosmosClient(URL, credential=KEY)
DATABASE_NAME = 'contoso-outdoor'
CONTAINER_NAME = 'customers'

In [ ]:
# Create the database if it doesnt already exist
client = CosmosClient(URL, credential=KEY)
try:
    database = client.create_database(DATABASE_NAME)
except exceptions.CosmosResourceExistsError:
    database = client.get_database_client(DATABASE_NAME)

print(database)

In [ ]:
# Create the container if it doesnt already exist
try:
    container = database.create_container(id=CONTAINER_NAME, partition_key=PartitionKey(path="/id"))
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(CONTAINER_NAME)
except exceptions.CosmosHttpResponseError:
    raise
print(container)

In [ ]:
# Loop through each json file in data/customer_info and insert into container
import os
import json
import glob
path = '.'
for filename in glob.glob(os.path.join(path, '*.json')):
    with open(filename) as file:
        customer = json.load(file)
        container.upsert_item(customer)
        print('Upserted item with id {0}'.format(customer['id']))

In [ ]:
# Get items from container to validate they were inserted
print('Get all items in container')
items = list(container.read_all_items(max_item_count=10))
print(items)
